In [1]:
import duckdb

In [2]:
con = duckdb.connect(database='dados.duckdb.db', read_only=False)

In [10]:
import duckdb
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

# Tente consultar a tabela agora
df = con.execute("select * from bronze_z0019").fetchdf()
df.head()

,NARBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,10001,PARAFUSO,BT10,100,150,None,NaT
1,10002,PORCA,BT20,100,250,None,NaT
2,10003,PREGO,BT30,100,50,None,NaT
3,10004,SERRA,BT40,100,200,None,NaT
4,10003,PREGO,BT90,100,50,z0019_2.csv,2025-12-04 16:45:24


In [15]:
df = con.execute(""" 
                 SELECT * 
                 FROM (
                    SELECT *, ROW_NUMBER() OVER(PARTITION BY NARBR ORDER BY data_ingestao DESC) AS row_num
                    FROM bronze_z0019 
                    WHERE data_ingestao >= '2025-01-11'
                 )WHERE row_num = 1
                    """).fetchdf()

df.head(10)

,NARBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row_num
0,10009,CHAVE DE FENDA,BT70,100,300,z0019_2.csv,2025-12-04 16:45:24,1
1,10005,MACHADO,BT60,100,100,z0019_2.csv,2025-12-04 16:45:24,1
2,10003,PREGO,BT90,100,50,z0019_2.csv,2025-12-04 16:45:24,1
3,10004,SERRA,BT50,100,200,z0019_2.csv,2025-12-04 16:45:24,1


In [17]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao','row_num'])
df_final = df_final.rename(columns={'NARBR': 'id'})
df_final = df_final.rename(columns={'MAKTX': 'nm_produto'})
df_final = df_final.rename(columns={'WERKS': 'id_categoria'})
df_final = df_final.rename(columns={'MAINS': 'id_fornecedor'})
df_final = df_final.rename(columns={'LABST': 'vl_preco'})
df_final.head(10)


,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10009,CHAVE DE FENDA,BT70,100,300
1,10005,MACHADO,BT60,100,100
2,10003,PREGO,BT90,100,50
3,10004,SERRA,BT50,100,200


In [21]:
df2 = df_final
df2 = df2.astype(
    {
        'id': int, 
        'nm_produto': str,
        'id_categoria': str, 
        'id_fornecedor': int, 
        'vl_preco': float
    }
)
df2.dtypes
#df_final.head(10)

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [23]:
con.execute(""" 
    CREATE TABLE IF NOT EXISTS produtos (
        id BIGINT,
        nm_produto TEXT,
        id_categoria TEXT,
        id_fornecedor BIGINT,
        vl_preco FLOAT
            )
    """)

In [24]:
df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10009,CHAVE DE FENDA,BT70,100,300.0
1,10005,MACHADO,BT60,100,100.0
2,10003,PREGO,BT90,100,50.0
3,10004,SERRA,BT50,100,200.0


In [26]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [27]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10009,CHAVE DE FENDA,BT70,100,300.0
1,10005,MACHADO,BT60,100,100.0
2,10003,PREGO,BT90,100,50.0
3,10004,SERRA,BT50,100,200.0


In [28]:
con.close()